In [13]:
import ollama
import httpx
import os
import json
from typing import Optional
from pydantic import BaseModel

from dotenv import load_dotenv
load_dotenv()


True

In [40]:
def ollama_query(user_prompt, system_prompt=None, temperature=0.8):
    messages = [
        {"role": "user", "content": user_prompt},
    ]
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    if temperature:
        options = {"temperature": temperature}
    response = ollama.chat(
        model="qwen2.5-coder:32b",
        messages=messages,
        options=options or {}
    )
    return response["message"]["content"]

test_ollama = ollama_query("Hello, this is a test.", temperature=0.1)
print(test_ollama)  

Hello! How can I assist you with your test? Feel free to ask any questions or let me know if you need help with anything specific.


In [8]:
db_url = f"{os.environ.get("AG3NTS_CENTRALA_URL")}/apidb"
api_key = os.environ.get("AG3NTS_API_KEY")

class ApiDbQuery(BaseModel):
    task: Optional[str] = "database"
    apikey: Optional[str] = api_key
    query: str

def query_db(query: str):
    """Send prompt to ApiDb"""
    data = ApiDbQuery(query=query)
    response = httpx.post(db_url, data=data.model_dump_json())
    return response.json()

def get_tables():
    query = "show tables"
    response = query_db(query)
    return [value for item in response["reply"] for value in item.values()]

def get_table_layout(name):
    query = f"show create table {name}"
    response = query_db(query)
    return {
        "name": name,
        "layout": response["reply"][0]["Create Table"]
    }

In [9]:
tables = get_tables()
layouts = [get_table_layout(table) for table in tables]

In [14]:
json.dumps(layouts)

'[{"name": "connections", "layout": "CREATE TABLE `connections` (\\n  `user1_id` int(11) NOT NULL,\\n  `user2_id` int(11) NOT NULL,\\n  PRIMARY KEY (`user1_id`,`user2_id`)\\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci"}, {"name": "correct_order", "layout": "CREATE TABLE `correct_order` (\\n  `base_id` int(11) DEFAULT NULL,\\n  `letter` char(1) DEFAULT NULL,\\n  `weight` int(11) DEFAULT 0\\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci"}, {"name": "datacenters", "layout": "CREATE TABLE `datacenters` (\\n  `dc_id` int(11) DEFAULT NULL,\\n  `location` varchar(30) NOT NULL,\\n  `manager` int(11) NOT NULL DEFAULT 31,\\n  `is_active` int(11) DEFAULT 0\\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci"}, {"name": "users", "layout": "CREATE TABLE `users` (\\n  `id` int(11) NOT NULL AUTO_INCREMENT,\\n  `username` varchar(20) DEFAULT NULL,\\n  `access_level` varchar(20) DEFAULT \'user\',\\n  `is_active` int(11) DEFAULT 1,\\n  `lastlo

In [41]:
system_prompt = f"""
You are a helpful SQL query expert.
Your first task is to analyze the database layout based on provided information.
The information format is not known in advance. 
The only thing we know is that it will be structured as:
{{"name": "table name", "layout": any information about the table}}
The information will be labeled as <layout> by the user.

Create a detailed summary of the database layout for your own reference.

After you have analyzed the layout, your task will be to prepare an SQL query that will return IDs of all datacenters
managed by managers who are currently on leave (their status is inactive).

The query should be based on the provided database layout.
It must take into account all kinds of relationship between tables - both explicitly defined and implicit.

Build your answer ONLY based on provided information. Do not make ANY additional assumption about database structure or possible data values.

After you have prepared the query, explain your reasoning in points and return the reasoning and the query.
"""

result = ollama_query(json.dumps(layouts), system_prompt, temperature=0.1)

In [42]:
from pprint import pprint

pprint(result)

('### Database Layout Summary\n'
 '\n'
 'Based on the provided information, here is a detailed summary of the '
 'database layout:\n'
 '\n'
 '1. **connections Table**\n'
 '   - This table represents connections between users.\n'
 '   - It has two columns: `user1_id` and `user2_id`, which are both integers '
 'and serve as the primary key.\n'
 '\n'
 '2. **correct_order Table**\n'
 '   - This table seems to be related to some kind of ordering or ranking '
 'system.\n'
 '   - It contains three columns:\n'
 '     - `base_id`: An integer that might reference another table (possibly '
 '`datacenters`).\n'
 '     - `letter`: A single character, possibly representing a category or '
 'type.\n'
 '     - `weight`: An integer that could represent the weight or priority of '
 'the entry.\n'
 '\n'
 '3. **datacenters Table**\n'
 '   - This table contains information about data centers.\n'
 '   - It has four columns:\n'
 '     - `dc_id`: An integer identifier for each data center.\n'
 '     - `locati

In [43]:
system_prompt = f"""
Your task is to parse the provided analysis and extract an SQL query from it.
Reformat the query to be more readable and return it. 
Return only the query, with no additional comments or explanations or formatting so that
it is directly usable for database operations.
I don't want the response to contain any newlines, tabs, or extra spaces or whitespace characters.
"""

clean_query = ollama_query(result, system_prompt)

In [44]:
qry = clean_query.strip("```").strip("sql")
qry

'SELECT dc_id FROM datacenters d JOIN users u ON d.manager = u.id WHERE u.is_active = 0'

In [53]:
answer = query_db(qry)
answer

{'reply': [{'dc_id': '4278'}, {'dc_id': '9294'}], 'error': 'OK'}

In [54]:
answer = answer["reply"]
answer

[{'dc_id': '4278'}, {'dc_id': '9294'}]

In [55]:
from aidevs3.poligon import send

load_dotenv()

key = os.environ.get("AG3NTS_API_KEY")
url = f"{os.environ.get("AG3NTS_CENTRALA_URL")}/report"

res = send(url, answer=answer, apikey=key, task="database")
print(res)

Exception: Failed to send data: {
    "code": -302,
    "message": "wrong answer"
}